# Notebook for testing MPS and MPU devices on Apple Silicon Macs

In [1]:
import platform
import sys
import pandas as pd
import numpy as np
import sklearn as sk
import scipy as sp
import skimage as ski
import tensorflow as tf
import keras
import torch


print(f"Python Version: {sys.version}")
print(f"Python Platform: {platform.platform ()}")
print()
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Scikit-Learn Version: {sk.__version__}")
print(f"Scikit-Image Version: {ski.__version__}")
print(f"SciPy Version: {sp.__version__}")
print()

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
# Check if GPU for Tensorflow/Keras is available
gpu = len(tf.config.list_physical_devices("GPU")) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
print()

print(f"PyTorch Version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device
# device = "mps" if torch.backends.mps.is_available() else "cpu"
device = (
    "mps"
    if torch.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

# from catboost.utils import get_gpu_device_count
# print("I see %i GPU devices" % get_gpu_device_count())

Python Version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:21:42) [Clang 18.1.8 ]
Python Platform: macOS-15.2-arm64-arm-64bit

Pandas Version: 2.2.3
NumPy Version: 1.26.4
Scikit-Learn Version: 1.5.2
Scikit-Image Version: 0.25.0
SciPy Version: 1.15.0

Tensor Flow Version: 2.17.1
Keras Version: 3.7.0
GPU is available

PyTorch Version: 2.5.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


## XGBoost

In [2]:
import xgboost

build_info = xgboost.build_info()
for name in sorted(build_info.keys()):
    print(f"{name}: {build_info[name]}")

BUILTIN_PREFETCH_PRESENT: True
CLANG_VERSION: [18, 1, 8]
DEBUG: False
MM_PREFETCH_PRESENT: False
USE_CUDA: False
USE_DLOPEN_NCCL: False
USE_FEDERATED: False
USE_NCCL: False
USE_OPENMP: True
USE_RMM: False
libxgboost: /opt/homebrew/Caskroom/miniforge/base/envs/helloDogBreed/lib/libxgboost.dylib


USE_CUDA: False -> needs rebuilding with CUDA support for macOS / Apple Silicon (arm64)

In [4]:
# Enable GPU acceleration
# model = xgboost.XGBClassifier(device="cuda")

# Disable GPU acceleration (use CPU)
model = xgboost.XGBClassifier(device="cpu")

In [6]:
xgboost?

Type:        module
String form: <module 'xgboost' from '/opt/homebrew/Caskroom/miniforge/base/envs/helloDogBreed/lib/python3.11/site-packages/xgboost/__init__.py'>
File:        /opt/homebrew/Caskroom/miniforge/base/envs/helloDogBreed/lib/python3.11/site-packages/xgboost/__init__.py
Docstring:  
XGBoost: eXtreme Gradient Boosting library.

Contributors: https://github.com/dmlc/xgboost/blob/master/CONTRIBUTORS.md

not working: [Enabling GPUXGBoost on Apple M1 Pro Chipset for Faster Training](https://devcodef1.com/news/1136152/gpuxgboost-on-m1-pro)

In [12]:
# !pwd

/Users/tillmeineke/ML/ML_Zoomcamp2024_hw/01-intro/setup


In [ ]:
# !git clone --recursive https://github.com/dmlc/xgboost
# !cd xgboost
# !pwd
# !ls

/Users/tillmeineke/ML/ML_Zoomcamp2024_hw/01-intro/setup
00_portfolio_page_github.md  apple_metal_test_env.ipynb
01_setup_environments.md     dotfiles_for_aws
03_setup_macBook.md          extensions.json
04_setup_vscode.md           images
05_repo.md                   weekly_learning.md
06_dotfiles.md               xgboost
07_neovim.md                 xgboost_GPU_ApppleSilicon.md
08_obsidian.md


In [16]:
# !cd xgboost && cp make/minimum.mk config.mksed -i '' 's/^CXX =.*/CXX = clang++/' config.mksed -i '' 's/^CC =.*/CC = clang/' config.mksed -i '' 's/^USE_OPENMP =.*/USE_OPENMP = 0/' config.mksed -i '' 's/^CMAKE_PREFIX_PATH =.*/CMAKE_PREFIX_PATH = /usr/local/' config.mksed -i '' 's/^CMAKE_OSX_SYSROOT =.*/CMAKE_OSX_SYSROOT = macosx/' config.mksed -i '' 's/^CMAKE_CXX_FLAGS =.*/CMAKE_CXX_FLAGS = -march=native -mtune=native -I/usr/local/include -I/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/' config.mksed -i '' 's/^CMAKE_C_FLAGS =.*/CMAKE_C_FLAGS = -march=native -mtune=native -I/usr/local/include -I/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/' config.mk

cp: config.mk is not a directory


In [ ]:
# !make -j$(sysctl -n hw.physicalcpu) USE_CUDA=1

In [ ]:
# !cd python-package; python3 setup.py install --use-cuda

above did not work. need to try [How to Install XGBoost for Python on macOS](https://machinelearningmastery.com/install-xgboost-python-macos/)

## Catboost

In CatBoost you can use the "task_type" property, for example, like this CatBoostRegressor(task_type="GPU"), and that's it. See the [documentation](https://catboost.ai/en/docs/features/training-on-gpu).

## Numpy configuration

In [2]:
# Show the configuration of NumPy
np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4377784592
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -ftree-vectorize, -fPIC, -fstack-protector-strong, -O2, -pipe, -isystem,
      /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include, -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/numpy_1707225640867/work=/usr/local/src/conda/numpy-1.26.4,
      -fdebug-prefix-map=/opt/homebrew/Caskroom/miniforge/base/envs/ML_Zo

## SciPy configuration

In [3]:
# Show the configuration of SciPy
sp.__config__.show()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    lib directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib
    name: lapack
    openblas configuration: unknown
    pc file directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/lib/pkgconfig
    version: 3.9.0
  pybind11:
    detection method: pkgconfig
    include directory: /opt/homebrew/Caskroom/miniforge/base/envs/ML_Zoomcamp2024/include
    name: pybind11
    version: 2.13.4
Compilers:
  c: